In [1]:
!pip install sentence-transformers faiss-cpu pandas joblib openpyxl ipywidgets tabulate --quiet

from sentence_transformers import SentenceTransformer
import pandas as pd, numpy as np, faiss, joblib
import ipywidgets as widgets
from IPython.display import display, clear_output
from tabulate import tabulate


# UPLOAD FILE DATASET

from google.colab import files
uploaded = files.upload()
dataset_path = list(uploaded.keys())[0]
print(f"Đã tải file dataset: {dataset_path}")


# ĐỌC DỮ LIỆU & CHUẨN HÓA

df = pd.read_excel(dataset_path)
print(" Các cột trong dataset:", list(df.columns))

# Thêm category_id nếu chưa có
if "category_id" not in df.columns:
    df.insert(0, "category_id", range(1, len(df) + 1))

# Gộp 3 cột summary thành 1
df_melted = df.melt(
    id_vars=["category_id", "category_name"],
    value_vars=["summary1", "summary2", "summary3"],
    var_name="summary_index",
    value_name="summary"
).dropna(subset=["summary"])

# Gộp text cho embedding (E5 khuyến nghị prefix "passage: ")
df_melted["text"] = "passage: " + df_melted["category_name"].astype(str) + ". " + df_melted["summary"].astype(str)
texts = df_melted["text"].tolist()

print(f" Tổng số dòng dữ liệu sau khi gộp: {len(texts)}")


# LOAD MÔ HÌNH & TẠO EMBEDDING
model_name = "intfloat/multilingual-e5-small"
model = SentenceTransformer(model_name)
print(f" Đã tải mô hình: {model_name}")

def encode_batched(model, items, batch_size=256):
    outs = []
    for i in range(0, len(items), batch_size):
        batch = items[i:i+batch_size]
        arr = model.encode(batch, convert_to_numpy=True, show_progress_bar=False)
        outs.append(arr.astype("float32"))
    return np.vstack(outs)

embeddings = encode_batched(model, texts, batch_size=256)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)  # normalize L2



# TẠO FAISS INDEX (Cosine similarity)

index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(np.array(embeddings).astype("float32"))
print(f"FAISS index tạo với {index.ntotal} vector (normalized).")


# GIAO DIỆN TEST (UI + Button)

def search(query, top_k=5):
    q = "query: " + query.strip()
    vec = model.encode([q], convert_to_numpy=True)
    vec = vec / np.linalg.norm(vec, axis=1, keepdims=True)
    D, I = index.search(vec.astype("float32"), k=top_k)

    rows = []
    for rank, idx in enumerate(I[0]):
        row = df_melted.iloc[idx]
        rows.append([
            int(row["category_id"]),
            row["category_name"],
            round(float(D[0][rank]), 3),
            row["summary"][:150] + "..."
        ])

    print(f"\n Kết quả cho truy vấn: '{query}'\n")
    print(tabulate(rows, headers=["ID", "Category", "Similarity", "Summary"], tablefmt="fancy_grid"))

# UI
query_box = widgets.Text(
    value='',
    placeholder='Nhập truy vấn tìm kiếm...',
    description=' Query:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)
search_button = widgets.Button(description="Tìm kiếm", button_style='success')

def on_search_click(b):
    clear_output(wait=True)
    display(query_box, search_button)
    if query_box.value.strip():
        search(query_box.value.strip())
    else:
        print(" Vui lòng nhập truy vấn!")

search_button.on_click(on_search_click)
display(query_box, search_button)


# LƯU BUNDLE
import os, joblib, faiss

os.makedirs("models", exist_ok=True)

# Chỉ giữ các cột cần cho API
metadata_for_api = df_melted[["category_id", "category_name", "summary"]].reset_index(drop=True)

bundle = {
    "model_name": model_name,
    "faiss_index": index,
    "metadata": metadata_for_api,
    "metric_type": faiss.METRIC_INNER_PRODUCT,
    "vectors_normalized": True
}

out_path = "models/semantic_search_model.pkl"
joblib.dump(bundle, out_path)
print(f" Đã lưu bundle: {out_path}")
print(f"   → ntotal={index.ntotal}, metric=METRIC_INNER_PRODUCT, normalized=True")

# (tuỳ chọn) Lưu luôn metadata đã melt để tái lập
metadata_for_api.to_csv("models/metadata_melted.csv", index=False, encoding="utf-8-sig")
print(" Đã lưu kèm: models/metadata_melted.csv")



Text(value='magic', description=' Query:', layout=Layout(width='80%'), placeholder='Nhập truy vấn tìm kiếm...'…

Button(button_style='success', description='Tìm kiếm', style=ButtonStyle())


 Kết quả cho truy vấn: 'magic'

╒══════╤══════════════════════╤══════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   ID │ Category             │   Similarity │ Summary                                                                                                                  │
╞══════╪══════════════════════╪══════════════╪══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│    3 │ Y tế - Sức khỏe      │        0.808 │ Phạm vi bao trùm nghiên cứu bệnh học, dinh dưỡng, phòng bệnh, vaccine và liệu pháp điều trị tiên tiến....                │
├──────┼──────────────────────┼──────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│   14 │ Tự động hóa          │        0.807 │ Quá trình sử dụng công nghệ và máy móc để thực hiện các công việc lặp 